In [1]:
import hopsworks
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
import seaborn as sns
from matplotlib import pyplot
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib
import os

/u/11/zhangz13/unix/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
api = 'HeCatNGJxisb99Vf.ircWdTrkgbbZpBMU7iPN2zqDIwoTuaSX88LPeISIMJHuzP3icXixNd6JFcWUqakL'
project = hopsworks.login(api_key_value = api)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/229529
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
wine_fg = fs.get_feature_group(name="wine_2", version=16)
query = wine_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="wine_2",
                                  version=16,
                                  description="Read from Wine Quality dataset",
                                  labels=["quality"],
                                  query=query)

In [4]:
# You can read training data, randomly split into train/test sets of features (X) and labels (y)        
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.2)
print("length of x y_train max:",y_train.max())
print("length of x X_train:",X_train.shape)
print("length of x X_test:",X_test.shape)
from sklearn.linear_model import LogisticRegression
# Train our model with the Scikit-learn K-nearest-neighbors algorithm using our features (X_train) and labels (y_train)
from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier(random_state = 0, n_estimators = 150,
#                                     criterion = 'entropy')
model = LogisticRegression(random_state = 0, penalty = 'l2',max_iter=15000 )
model.fit(X_train, y_train.values.ravel())

Finished: Reading data from Hopsworks, using ArrowFlight (1.37s) 


length of x y_train max: quality    2
dtype: int64
length of x X_train: (4262, 11)
length of x X_test: (1066, 11)


LogisticRegression(max_iter=15000, random_state=0)

In [5]:
X_train

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol
0,10.100000,0.310000,0.350000,1.600000,0.075000,9.000000,28.000000,0.996720,3.240000,0.830000,11.200000
1,9.000000,0.800000,0.120000,2.400000,0.083000,8.000000,28.000000,0.998360,3.330000,0.650000,10.400000
2,12.200000,0.480000,0.540000,2.600000,0.085000,19.000000,64.000000,1.000000,3.100000,0.610000,10.500000
4,6.400000,0.220000,0.320000,7.900000,0.029000,34.000000,124.000000,0.994800,3.400000,0.390000,10.200000
5,5.000000,0.255000,0.220000,2.700000,0.043000,46.000000,153.000000,0.992380,3.750000,0.760000,11.300000
...,...,...,...,...,...,...,...,...,...,...,...
5323,7.000000,0.230000,0.420000,5.100000,0.042000,37.000000,144.000000,0.995180,3.500000,0.590000,10.200000
5324,6.000000,0.280000,0.270000,4.100000,0.046000,50.000000,147.000000,0.991260,3.270000,0.560000,11.600000
5325,7.100000,0.310000,0.500000,14.500000,0.059000,6.000000,148.000000,0.998300,2.940000,0.440000,9.100000
5326,7.100000,0.670000,0.000000,2.300000,0.083000,18.000000,27.000000,0.997680,3.440000,0.540000,9.400000


In [6]:
y_train.value_counts()

quality
1          2540
0          1597
2           125
Name: count, dtype: int64

In [7]:
y_test.value_counts()

quality
1          644
0          393
2           29
Name: count, dtype: int64

In [8]:
y_pred = model.predict(X_test)
y_pred.min()

0

In [11]:
## Logistic Regression
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0, penalty = 'l2',max_iter=15000 )
classifier.fit(X_train, y_train)

# Predicting Test Set
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='weighted')
rec = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

results = pd.DataFrame([['Logistic Regression', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])
print(results)

                 Model  Accuracy  Precision    Recall  F1 Score
0  Logistic Regression  0.748593    0.72613  0.748593  0.734581


In [12]:
from sklearn.neural_network import MLPClassifier
classifier = MLPClassifier(hidden_layer_sizes=(50,100,200),  
                         max_iter=20000,)
classifier.fit(X_train, y_train)

# Predicting Test Set
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='weighted')
rec = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

results = pd.DataFrame([['Logistic Regression', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])
print(results)

                 Model  Accuracy  Precision    Recall  F1 Score
0  Logistic Regression  0.729831    0.70695  0.729831  0.716246


In [14]:
## Randomforest
from sklearn.ensemble import RandomForestClassifier
import numpy as np
model = RandomForestClassifier(random_state = 0, n_estimators = 150,
                                    criterion = 'entropy')
model.fit(X_train, y_train)

# Predicting Test Set
y_pred = model.predict(X_test)
print("y_pred max is:",y_train.max())
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='weighted')
rec = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

model_results = pd.DataFrame([['Random Forest (n=100)', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])


print(model_results)

y_pred max is: quality    2
dtype: int64
                   Model  Accuracy  Precision    Recall  F1 Score
0  Random Forest (n=100)  0.763602   0.741158  0.763602  0.750966


In [10]:
# from sklearn.ensemble import RandomForestClassifier
# classifier = RandomForestClassifier(random_state = 0, n_estimators = 150,
#                                     criterion = 'entropy')
# y_train_1d = y_train.values.ravel() if isinstance(y_train, pd.DataFrame) else y_train.ravel()
# y_test_1d = y_test.values.ravel() if isinstance(y_test, pd.DataFrame) else y_test.ravel()

# classifier.fit(X_train, y_train)

# # Predicting Test Set
# y_pred = classifier.predict(X_test)
# print("y_pred max is:",y_train.max())
# acc = accuracy_score(y_test, y_pred)
# prec = precision_score(y_test, y_pred,average='weighted')
# rec = recall_score(y_test, y_pred,average='weighted')
# f1 = f1_score(y_test, y_pred,average='weighted')

# model_results = pd.DataFrame([['Random Forest (n=100)', acc, prec, rec, f1]],
#                columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])


# print(model_results)

y_pred max is: quality    2
dtype: int64
                   Model  Accuracy  Precision    Recall  F1 Score
0  Random Forest (n=100)  0.730769   0.712958  0.730769  0.721716


In [15]:
# Evaluate model performance using the features from the test set (X_test)
y_pred = model.predict(X_test)

# Compare predictions (y_pred) with the labels in the test set (y_test)
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
acc = accuracy_score(y_test, y_pred)
print(acc)

0.7636022514071295


In [16]:
# We will now upload our model to the Hopsworks Model Registry. First get an object for the model registry.
mr = project.get_model_registry()

# The contents of the 'iris_model' directory will be saved to the model registry. Create the dir, first.
model_dir="wine_model_2"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save both our model and the confusion matrix to 'model_dir', whose contents will be uploaded to the model registry
joblib.dump(model, model_dir + "/wine_model_3.pkl")
# fig.savefig(model_dir + "/confusion_matrix.png")    

# Specify the schema of the model's input/output using the features (X_train) and labels (y_train)
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)
# Ensure that the metrics are single numerical values
wine_model = mr.python.create_model(
    name="wine_model_3", 
    metrics={
        "accuracy": model_results['Accuracy'].iloc[0],
        "Precision": model_results['Precision'].iloc[0],
        "Recall": model_results['Recall'].iloc[0],
        "F1_score": model_results['F1 Score'].iloc[0]
    },
    model_schema=model_schema,
    description="Wine Quality Predictor"
)

# Upload the model to the model registry, including all files in 'model_dir'
wine_model.save(model_dir)


Connected. Call `.close()` to terminate connection gracefully.


Uploading: 100.000%|██████████| 1663/1663 elapsed<00:01 remaining<00:00:01,  2.85it/s]
Uploading: 100.000%|██████████| 17248097/17248097 elapsed<00:15 remaining<00:0085it/s]
Uploading: 100.000%|██████████| 1001/1001 elapsed<00:01 remaining<00:000:41, 10.36s/it]
Model export complete: 100%|██████████| 6/6 [00:25<00:00,  4.22s/it]                   

Model created, explore it at https://c.app.hopsworks.ai:443/p/229529/models/wine_model_3/1


Model(name: 'wine_model_3', version: 1)